In [55]:
import os
import re
from ipywidgets import Checkbox, Text, VBox, HBox, Layout, Output

In [2]:
dir_path = None

def get_path():
    import tkinter as tk
    from tkinter import filedialog
    global dir_path
    if dir_path == None:
        tk.Tk().withdraw()
        dir_path = filedialog.askdirectory()
    return dir_path

def find_mod_name(path):
    if path.isspace() or not os.path.isdir(path):
        raise Exception("Not Found")
        
    tail,head = os.path.split(path)
    if head=='mods': return 'mods'
    
    parent = find_mod_name(tail)
    #print('\t\tParent returned: ', parent)
    if parent == 'mods': return head
    else: return parent

def clean(s): return re.sub('\W|^(?=\d)','_', s).lower()

def neat_trim(s, ideal_length, max_length):
    # Shortcut for strings that are split by an emdash.
    clauses = re.split(' - |[(:]', s)
    if len(clauses[0]) > max_length/2 and len(clauses[0]) < max_length: return clauses[0]
    
    # If the string is short enough, return it unmodified.
    if len(s) < max_length: return s
    
    # Split the string at the ideal length.
    front = s[:ideal_length]
    back = s[ideal_length:]
    
    # If there's nothing in the back half, return the front.
    # This should never happen!
    if back == None or back.isspace(): return front
    
    # Split the string into words.
    words = back.split(' ')
    
    # The length of the word that can be added to the end without exceeding max_length.
    grace = max_length - ideal_length - 1
        
    #print('Front = "' + front + '", words = ' + str(words))
    
    # If the split didn't produce anything, return the front.
    # This should never happen!
    if words == None or len(words) == 0: return front

    # If the first word is too long, return the front with an ellipsis ...
    elif len(words[0]) > grace: return front + '\u2026'
    
    # Everything is fine so return the front with the first appended.
    else: return f'{front}{words[0]}'


In [7]:
mods = generate_list()

In [105]:
def generate_list():
    import glob
    mods = {}
    
    for root, d_names, f_names in os.walk(get_path()):   
        path = os.path.split(root)
        directory_name = path[-1] 

        if directory_name.lower() == 'slidersets':
            mod_name = 'invalid'        
            try:
                mod_name = find_mod_name(root).lower()
            except Exception as ex:
                print(path, ex)
                continue

            #print(mod_name)
            #print('\t' + root)
            #print('\t' + str(f_names))
    
            mods[mod_name] = []
        
            slider_set_glob = os.path.join(root, '*.osp')
            #for slider_set in glob.glob(slider_set_glob, recursive=True):
                #print('\t\t' + slider_set)
                #slider_set = d_name
                #condition_filename = os.path.join(group_path, '_conditions.txt')
                
    return mods

def generate_checklist(mods):
    out = Output()
    checkboxes = [Checkbox(value=False, description = mod, layout=Layout(width='100%')) for mod in mods.keys()]                  
    checkboxes_box = VBox(checkboxes, layout=Layout(width='45%'))
    checkedboxes_box = VBox(layout=Layout(width='45%'))

    @out.capture()
    def checkbox_disabler(b):
        cb = b.owner
        if cb.value:
            if cb in checkboxes_box.children: checkboxes_box.children = tuple(x for x in checkboxes_box.children if x != cb)
            if cb not in checkedboxes_box.children: checkedboxes_box.children = checkedboxes_box.children + (cb,)
        else:
            if cb in checkedboxes_box.children: checkedboxes_box.children = tuple(x for x in checkedboxes_box.children if x != cb)
            if cb not in checkboxes_box.children: checkboxes_box.children = checkboxes_box.children + (cb,)

    for cb in checkboxes: cb.observe(checkbox_disabler)
    
    display(HBox([checkboxes_box, checkedboxes_box], layout=Layout(width='90%')))
    

In [106]:
generate_checklist(mods)